In [1]:
import pandas as pd
import os
import numpy as np
from gensim.models import Word2Vec, FastText
# import glove nitin commented as its not used
# from glove import Corpus

import collections
import gc 

import keras
from keras import backend as K
from keras import regularizers
from keras.models import Sequential, Model
from keras.layers import Flatten, Dense, Dropout, Input, concatenate, merge, Activation, Concatenate, LSTM, GRU
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Conv1D, BatchNormalization, GRU, Convolution1D, LSTM
from keras.layers import UpSampling1D, MaxPooling1D, GlobalMaxPooling1D, GlobalAveragePooling1D,MaxPool1D, merge

# from keras.optimizers import Adam # nitin commented as Adam has been shifted to optimizer_v1 module.
from tensorflow.keras.optimizers import Adam

from keras.callbacks import EarlyStopping, ModelCheckpoint, History, ReduceLROnPlateau
from keras.utils import np_utils
#from keras.backend.tensorflow_backend import set_session, clear_session, get_session # nitin commented as tensorflow_backend not used anymore
from keras.backend import set_session, clear_session, get_session

import tensorflow as tf


from sklearn.utils import class_weight
from sklearn.metrics import average_precision_score, roc_auc_score, accuracy_score, f1_score

import warnings
warnings.filterwarnings('ignore')

In [2]:
# Reset Keras Session
def reset_keras(model):
    sess = get_session()
    clear_session()
    sess.close()
    sess = get_session()

    try:
        del model # this is from global space - change this as you need
    except:
        pass

    gc.collect() # if it's done something you should see a number being outputted

def make_prediction_timeseries(model, test_data):
    probs = model.predict(test_data)
    y_pred = [1 if i>=0.5 else 0 for i in probs]
    return probs, y_pred

def save_scores_timeseries(predictions, probs, ground_truth, model_name, 
                problem_type, iteration, hidden_unit_size, type_of_ner):
    
    auc = roc_auc_score(ground_truth, probs)
    auprc = average_precision_score(ground_truth, probs)
    acc   = accuracy_score(ground_truth, predictions)
    F1    = f1_score(ground_truth, predictions)
    
    
    result_dict = {}    
    result_dict['auc'] = auc
    result_dict['auprc'] = auprc
    result_dict['acc'] = acc
    result_dict['F1'] = F1

        
    file_name = str(hidden_unit_size)+"-"+model_name+"-"+problem_type+"-"+str(iteration)+"-"+type_of_ner+".p"
    
    result_path = "results/"
    print('auc{0}, auprc{1}, acc{2}, F1{3} '.format(auc,auprc,acc,F1))
    pd.to_pickle(result_dict, os.path.join(result_path, file_name))

    print(auc, auprc, acc, F1)

In [3]:
def timeseries_model(layer_name, number_of_unit):
    K.clear_session()
    
    sequence_input = Input(shape=(24,104),  name = "timeseries_input")
    
    if layer_name == "LSTM":
        x = LSTM(number_of_unit)(sequence_input)
    else:
        x = GRU(number_of_unit)(sequence_input)
    
    # nitin removed the tf.contrib with tf.compat.v1.estimator 
    
    #logits_regularizer = tf.compat.v1.estimator.layers.l2_regularizer(scale=0.01)
    # removed tf.compat.v1.estimator.layers.xavier_initializer() with glorot_normal
    logits_regularizer = tf.keras.regularizers.L2(0.01)
    
    sigmoid_pred = Dense(1, activation='sigmoid',use_bias=False,
                         kernel_initializer=tf.keras.initializers.glorot_normal(), 
                  kernel_regularizer=logits_regularizer)(x)
    
    
    model = Model(inputs=sequence_input, outputs=sigmoid_pred)
    
    
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
    return model

In [4]:
type_of_ner = "new"

x_train_lstm = pd.read_pickle("data/"+type_of_ner+"_x_train.pkl")
x_dev_lstm = pd.read_pickle("data/"+type_of_ner+"_x_dev.pkl")
x_test_lstm = pd.read_pickle("data/"+type_of_ner+"_x_test.pkl")

y_train = pd.read_pickle("data/"+type_of_ner+"_y_train.pkl")
y_dev = pd.read_pickle("data/"+type_of_ner+"_y_dev.pkl")
y_test = pd.read_pickle("data/"+type_of_ner+"_y_test.pkl")

In [5]:
epoch_num = 100
model_patience = 3
monitor_criteria = 'val_loss'
batch_size = 128

unit_sizes = [128, 256]
#unit_sizes = [256]
iter_num = 11
target_problems = ['mort_hosp', 'mort_icu', 'los_3', 'los_7']
layers = ["LSTM", "GRU"]
#layers = ["GRU"]
for each_layer in layers:
    print("Layer: ", each_layer)
    for each_unit_size in unit_sizes:
        print("Hidden unit: ", each_unit_size)
        for iteration in range(1, iter_num):
            print("Iteration number: ", iteration)
            print("=============================")

            for each_problem in target_problems:
                print ("Problem type: ", each_problem)
                print ("__________________")


                early_stopping_monitor = EarlyStopping(monitor=monitor_criteria, patience=model_patience)
                best_model_name = str(each_layer)+"-"+str(each_unit_size)+"-"+str(each_problem)+"-"+"best_model.hdf5"
                checkpoint = ModelCheckpoint(best_model_name, monitor='val_loss', verbose=1,
                    save_best_only=True, mode='min', period=1)


                callbacks = [early_stopping_monitor, checkpoint]

                model = timeseries_model(each_layer, each_unit_size)
                model.fit(x_train_lstm, y_train[each_problem], epochs=epoch_num, verbose=1, 
                          validation_data=(x_dev_lstm, y_dev[each_problem]), callbacks=callbacks, batch_size= batch_size)

                model.load_weights(best_model_name)

                probs, predictions = make_prediction_timeseries(model, x_test_lstm)
                save_scores_timeseries(predictions, probs, y_test[each_problem].values,str(each_layer),
                                       each_problem, iteration, each_unit_size,type_of_ner)
                reset_keras(model)
                #del model
                clear_session()
                gc.collect()

Layer:  LSTM
Hidden unit:  128
Iteration number:  1
Problem type:  mort_hosp
__________________
Epoch 1/100
119/119 [==============================] - ETA: 0s - loss: 0.3105 - acc: 0.8914
Epoch 00001: val_loss improved from inf to 0.25147, saving model to LSTM-128-mort_hosp-best_model.hdf5
119/119 [==============================] - 7s 44ms/step - loss: 0.3105 - acc: 0.8914 - val_loss: 0.2515 - val_acc: 0.9177
Epoch 2/100
118/119 [============================>.] - ETA: 0s - loss: 0.2378 - acc: 0.9181
Epoch 00002: val_loss improved from 0.25147 to 0.25075, saving model to LSTM-128-mort_hosp-best_model.hdf5
119/119 [==============================] - 5s 40ms/step - loss: 0.2378 - acc: 0.9180 - val_loss: 0.2508 - val_acc: 0.9104
Epoch 3/100
119/119 [==============================] - ETA: 0s - loss: 0.2160 - acc: 0.9261
Epoch 00003: val_loss improved from 0.25075 to 0.24512, saving model to LSTM-128-mort_hosp-best_model.hdf5
119/119 [==============================] - 5s 38ms/step - loss: 0.2

auc0.7242385051000878, auprc0.19961193901595387, acc0.9174333026678932, F10.042666666666666665 
0.7242385051000878 0.19961193901595387 0.9174333026678932 0.042666666666666665
Iteration number:  2
Problem type:  mort_hosp
__________________
Epoch 1/100
119/119 [==============================] - ETA: 0s - loss: 0.3053 - acc: 0.8890
Epoch 00001: val_loss improved from inf to 0.25140, saving model to LSTM-128-mort_hosp-best_model.hdf5
119/119 [==============================] - 16s 103ms/step - loss: 0.3053 - acc: 0.8890 - val_loss: 0.2514 - val_acc: 0.9117
Epoch 2/100
119/119 [==============================] - ETA: 0s - loss: 0.2354 - acc: 0.9161
Epoch 00002: val_loss improved from 0.25140 to 0.24383, saving model to LSTM-128-mort_hosp-best_model.hdf5
119/119 [==============================] - 12s 99ms/step - loss: 0.2354 - acc: 0.9161 - val_loss: 0.2438 - val_acc: 0.9122
Epoch 3/100
119/119 [==============================] - ETA: 0s - loss: 0.2145 - acc: 0.9232
Epoch 00003: val_loss impro

Epoch 6/100
119/119 [==============================] - ETA: 0s - loss: 0.1906 - acc: 0.9376
Epoch 00006: val_loss did not improve from 0.26022
119/119 [==============================] - 13s 107ms/step - loss: 0.1906 - acc: 0.9376 - val_loss: 0.2790 - val_acc: 0.9154
auc0.71313342978091, auprc0.18229353480521857, acc0.9174333026678932, F10.03234501347708895 
0.71313342978091 0.18229353480521857 0.9174333026678932 0.03234501347708895
Iteration number:  3
Problem type:  mort_hosp
__________________
Epoch 1/100
119/119 [==============================] - ETA: 0s - loss: 0.3069 - acc: 0.8899
Epoch 00001: val_loss improved from inf to 0.25477, saving model to LSTM-128-mort_hosp-best_model.hdf5
119/119 [==============================] - 30s 135ms/step - loss: 0.3069 - acc: 0.8899 - val_loss: 0.2548 - val_acc: 0.9122
Epoch 2/100
119/119 [==============================] - ETA: 0s - loss: 0.2400 - acc: 0.9165
Epoch 00002: val_loss improved from 0.25477 to 0.24259, saving model to LSTM-128-mort_ho

Epoch 5/100
119/119 [==============================] - ETA: 0s - loss: 0.2084 - acc: 0.9326
Epoch 00005: val_loss did not improve from 0.25468
119/119 [==============================] - 14s 116ms/step - loss: 0.2084 - acc: 0.9326 - val_loss: 0.2593 - val_acc: 0.9224
Epoch 6/100
119/119 [==============================] - ETA: 0s - loss: 0.1933 - acc: 0.9373- ETA: 1s - loss: 0.
Epoch 00006: val_loss did not improve from 0.25468
119/119 [==============================] - 14s 121ms/step - loss: 0.1933 - acc: 0.9373 - val_loss: 0.2733 - val_acc: 0.9205
Epoch 7/100
119/119 [==============================] - ETA: 0s - loss: 0.1753 - acc: 0.9430- ETA: 7s - loss:  - ETA: 5s - loss: 0.1677 - - ETA: 4s - loss: 0.1686 - ETA: 3s  - ETA: 0s - loss: 0.1745 - acc
Epoch 00007: val_loss did not improve from 0.25468
119/119 [==============================] - 11s 96ms/step - loss: 0.1753 - acc: 0.9430 - val_loss: 0.2807 - val_acc: 0.9127
auc0.7140182753146743, auprc0.17977918511932156, acc0.91260349586016

119/119 [==============================] - ETA: 0s - loss: 0.2553 - acc: 0.9221
Epoch 00002: val_loss improved from 0.26137 to 0.25946, saving model to LSTM-128-los_7-best_model.hdf5
119/119 [==============================] - 13s 110ms/step - loss: 0.2553 - acc: 0.9221 - val_loss: 0.2595 - val_acc: 0.9242
Epoch 3/100
119/119 [==============================] - ETA: 0s - loss: 0.2417 - acc: 0.9238
Epoch 00003: val_loss did not improve from 0.25946
119/119 [==============================] - 13s 108ms/step - loss: 0.2417 - acc: 0.9238 - val_loss: 0.2603 - val_acc: 0.9228
Epoch 4/100
119/119 [==============================] - ETA: 0s - loss: 0.2277 - acc: 0.9271
Epoch 00004: val_loss did not improve from 0.25946
119/119 [==============================] - 13s 106ms/step - loss: 0.2277 - acc: 0.9271 - val_loss: 0.2626 - val_acc: 0.9224
Epoch 5/100
119/119 [==============================] - ETA: 0s - loss: 0.2106 - acc: 0.9321- ETA: 6
Epoch 00005: val_loss did not improve from 0.25946
119/119 

...

119/119 [==============================] - ETA: 0s - loss: 0.2478 - acc: 0.9179
Epoch 00001: val_loss improved from inf to 0.18908, saving model to GRU-256-mort_icu-best_model.hdf5
119/119 [==============================] - 16s 113ms/step - loss: 0.2478 - acc: 0.9179 - val_loss: 0.1891 - val_acc: 0.9418
Epoch 2/100
119/119 [==============================] - ETA: 0s - loss: 0.1828 - acc: 0.9401
Epoch 00002: val_loss improved from 0.18908 to 0.17909, saving model to GRU-256-mort_icu-best_model.hdf5
119/119 [==============================] - 14s 120ms/step - loss: 0.1828 - acc: 0.9401 - val_loss: 0.1791 - val_acc: 0.9445
Epoch 3/100
119/119 [==============================] - ETA: 0s - loss: 0.1690 - acc: 0.9434
Epoch 00003: val_loss improved from 0.17909 to 0.16756, saving model to GRU-256-mort_icu-best_model.hdf5
119/119 [==============================] - 15s 123ms/step - loss: 0.1690 - acc: 0.9434 - val_loss: 0.1676 - val_acc: 0.9427
Epoch 4/100
119/119 [==============================] 

119/119 [==============================] - ETA: 0s - loss: 0.1958 - acc: 0.9314
Epoch 00006: val_loss did not improve from 0.23180
119/119 [==============================] - 13s 106ms/step - loss: 0.1958 - acc: 0.9314 - val_loss: 0.2445 - val_acc: 0.9122
auc0.8793508789320645, auprc0.5662319520080081, acc0.9142134314627415, F10.42170542635658914 
0.8793508789320645 0.5662319520080081 0.9142134314627415 0.42170542635658914
Problem type:  mort_icu
__________________
Epoch 1/100
119/119 [==============================] - ETA: 0s - loss: 0.2510 - acc: 0.9160
Epoch 00001: val_loss improved from inf to 0.18236, saving model to GRU-256-mort_icu-best_model.hdf5
119/119 [==============================] - 15s 109ms/step - loss: 0.2510 - acc: 0.9160 - val_loss: 0.1824 - val_acc: 0.9413
Epoch 2/100
119/119 [==============================] - ETA: 0s - loss: 0.1824 - acc: 0.9410
Epoch 00002: val_loss improved from 0.18236 to 0.17764, saving model to GRU-256-mort_icu-best_model.hdf5
119/119 [========

Epoch 2/100
119/119 [==============================] - ETA: 0s - loss: 0.2497 - acc: 0.9120
Epoch 00002: val_loss improved from 0.25253 to 0.23954, saving model to GRU-256-mort_hosp-best_model.hdf5
119/119 [==============================] - 14s 118ms/step - loss: 0.2497 - acc: 0.9120 - val_loss: 0.2395 - val_acc: 0.9150
Epoch 3/100
119/119 [==============================] - ETA: 0s - loss: 0.2316 - acc: 0.9181
Epoch 00003: val_loss improved from 0.23954 to 0.23682, saving model to GRU-256-mort_hosp-best_model.hdf5
119/119 [==============================] - 14s 117ms/step - loss: 0.2316 - acc: 0.9181 - val_loss: 0.2368 - val_acc: 0.9122
Epoch 4/100
119/119 [==============================] - ETA: 0s - loss: 0.2200 - acc: 0.9208
Epoch 00004: val_loss improved from 0.23682 to 0.23086, saving model to GRU-256-mort_hosp-best_model.hdf5
119/119 [==============================] - 14s 120ms/step - loss: 0.2200 - acc: 0.9208 - val_loss: 0.2309 - val_acc: 0.9168
Epoch 5/100
119/119 [=============

119/119 [==============================] - ETA: 0s - loss: 0.2345 - acc: 0.9234
Epoch 00005: val_loss did not improve from 0.25495
119/119 [==============================] - 13s 109ms/step - loss: 0.2345 - acc: 0.9234 - val_loss: 0.2551 - val_acc: 0.9219
Epoch 6/100
119/119 [==============================] - ETA: 0s - loss: 0.2229 - acc: 0.9273
Epoch 00006: val_loss did not improve from 0.25495
119/119 [==============================] - 13s 108ms/step - loss: 0.2229 - acc: 0.9273 - val_loss: 0.2643 - val_acc: 0.9219
auc0.730496808223188, auprc0.21487480940889753, acc0.9162833486660533, F10.07614213197969544 
0.730496808223188 0.21487480940889753 0.9162833486660533 0.07614213197969544
Iteration number:  10
Problem type:  mort_hosp
__________________
Epoch 1/100
119/119 [==============================] - ETA: 0s - loss: 0.3073 - acc: 0.8889
Epoch 00001: val_loss improved from inf to 0.24998, saving model to GRU-256-mort_hosp-best_model.hdf5
119/119 [==============================] - 16s 

Problem type:  los_7
__________________
Epoch 1/100
119/119 [==============================] - ETA: 0s - loss: 0.3220 - acc: 0.8987
Epoch 00001: val_loss improved from inf to 0.26549, saving model to GRU-256-los_7-best_model.hdf5
119/119 [==============================] - 18s 127ms/step - loss: 0.3220 - acc: 0.8987 - val_loss: 0.2655 - val_acc: 0.9224
Epoch 2/100
119/119 [==============================] - ETA: 0s - loss: 0.2600 - acc: 0.9208
Epoch 00002: val_loss improved from 0.26549 to 0.26117, saving model to GRU-256-los_7-best_model.hdf5
119/119 [==============================] - 14s 117ms/step - loss: 0.2600 - acc: 0.9208 - val_loss: 0.2612 - val_acc: 0.9233
Epoch 3/100
119/119 [==============================] - ETA: 0s - loss: 0.2481 - acc: 0.9228
Epoch 00003: val_loss improved from 0.26117 to 0.25886, saving model to GRU-256-los_7-best_model.hdf5
119/119 [==============================] - 14s 117ms/step - loss: 0.2481 - acc: 0.9228 - val_loss: 0.2589 - val_acc: 0.9242
Epoch 4/10